In [2]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [3]:
df = pd.read_csv(Path('./data/20210508.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice]
0,04/05/2021 20:57:38,Sixteen Candles (1984),The Battle of Algiers (1966),Brother (1997),A Clockwork Orange (1971),MONTAGE,[Default to Nicolas Cage],BRAIN: FRIEND OR FOE?,CUCKED BRITS,GANBARE NIPPON!
1,04/05/2021 21:06:42,Brother (1997),Sixteen Candles (1984),NaN,NaN,CUCKED BRITS,BRAIN: FRIEND OR FOE?,MONTAGE,GANBARE NIPPON!,[Default to Nicolas Cage]
2,04/05/2021 21:07:51,A Clockwork Orange (1971),The Battle of Algiers (1966),Brother (1997),Sixteen Candles (1984),BRAIN: FRIEND OR FOE?,GANBARE NIPPON!,CUCKED BRITS,MONTAGE,[Default to Nicolas Cage]
3,04/05/2021 21:49:59,Sixteen Candles (1984),The Battle of Algiers (1966),A Clockwork Orange (1971),Brother (1997),CUCKED BRITS,MONTAGE,BRAIN: FRIEND OR FOE?,[Default to Nicolas Cage],GANBARE NIPPON!
4,05/05/2021 08:54:20,Brother (1997),The Battle of Algiers (1966),NaN,NaN,GANBARE NIPPON!,CUCKED BRITS,[Default to Nicolas Cage],BRAIN: FRIEND OR FOE?,MONTAGE
5,05/05/2021 11:55:39,Brother (1997),The Battle of Algiers (1966),A Clockwork Orange (1971),Sixteen Candles (1984),MONTAGE,[Default to Nicolas Cage],BRAIN: FRIEND OR FOE?,GANBARE NIPPON!,CUCKED BRITS
6,05/05/2021 12:36:21,Brother (1997),The Battle of Algiers (1966),A Clockwork Orange (1971),NaN,CUCKED BRITS,BRAIN: FRIEND OR FOE?,GANBARE NIPPON!,MONTAGE,[Default to Nicolas Cage]


In [4]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [5]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Sixteen Candles (1984)', 'The Battle of Algiers (1966)',
       'Brother (1997)', 'A Clockwork Orange (1971)'], dtype=object)

In [6]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [7]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [8]:
submission_dict

{'Sixteen Candles (1984)': <Candidate('Sixteen Candles (1984)')>,
 'The Battle of Algiers (1966)': <Candidate('The Battle of Algiers (1966)')>,
 'Brother (1997)': <Candidate('Brother (1997)')>,
 'A Clockwork Orange (1971)': <Candidate('A Clockwork Orange (1971)')>}

In [9]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [10]:
ballots

[<Ballot(Sixteen Candles (1984), The Battle of Algiers (1966), Brother (1997), A Clockwork Orange (1971))>,
 <Ballot(Brother (1997), Sixteen Candles (1984))>,
 <Ballot(A Clockwork Orange (1971), The Battle of Algiers (1966), Brother (1997), Sixteen Candles (1984))>,
 <Ballot(Sixteen Candles (1984), The Battle of Algiers (1966), A Clockwork Orange (1971), Brother (1997))>,
 <Ballot(Brother (1997), The Battle of Algiers (1966))>,
 <Ballot(Brother (1997), The Battle of Algiers (1966), A Clockwork Orange (1971), Sixteen Candles (1984))>,
 <Ballot(Brother (1997), The Battle of Algiers (1966), A Clockwork Orange (1971))>]

In [11]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [12]:
print(election_result)

FINAL RESULT
Candidate                       Votes  Status
----------------------------  -------  --------
Brother (1997)                      4  Elected
Sixteen Candles (1984)              2  Rejected
A Clockwork Orange (1971)           1  Rejected
The Battle of Algiers (1966)        0  Rejected



In [13]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                    Votes  Status
-------------------------  -------  --------
CUCKED BRITS                     3  Hopeful
MONTAGE                          2  Hopeful
BRAIN: FRIEND OR FOE?            1  Rejected
GANBARE NIPPON!                  1  Rejected
[Default to Nicolas Cage]        0  Rejected

FINAL RESULT
Candidate                    Votes  Status
-------------------------  -------  --------
CUCKED BRITS                     5  Elected
MONTAGE                          2  Rejected
BRAIN: FRIEND OR FOE?            0  Rejected
GANBARE NIPPON!                  0  Rejected
[Default to Nicolas Cage]        0  Rejected

